# VGG19 template:

#### import libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_io as tfio
import os
from keras.utils import  to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input

In [ ]:
from keras import Sequential
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,GlobalAveragePooling2D
from sklearn.metrics import accuracy_score, confusion_matrix

#### import dataset

In [ ]:
d = tf.keras.utils.image_dataset_from_directory('IMGS', batch_size=10)

In [ ]:
data = d.map(lambda x,y: (x/255,y))

In [ ]:
data.as_numpy_iterator().next() 

#### define train, val, test

In [ ]:
train_size = int(len(data)*.7) 
val_size = int(len(data)*.2)  
test_size = int(len(data)*.1) 

In [ ]:
train = data.take(train_size)  #2110 images
val = data.skip(train_size).take(val_size) #600
test = data.skip(train_size+val_size).take(test_size) #300

In [ ]:
len(train)

In [ ]:
Y_train = np.concatenate([y for x, y in train], axis=0)
X_train = np.concatenate([x for x, y in train], axis=0)

Y_val = np.concatenate([y for x, y in val], axis=0)
X_val = np.concatenate([x for x, y in val], axis=0)

Y_test = np.concatenate([y for x, y in test], axis=0)
X_test = np.concatenate([x for x, y in test], axis=0)

In [ ]:
Y_train_cat = to_categorical(Y_train)
Y_val_cat = to_categorical(Y_val)
Y_test_cat = to_categorical(Y_test)

# VGG19

In [ ]:
from keras.applications.vgg19 import VGG19
vgg19_model = VGG19(weights='imagenet',include_top=False)

In [ ]:
x=vgg19_model.output

In [ ]:
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
x = Dense(512,activation='relu')(x) 
x = Dense(216,activation='relu')(x) 
preds = Dense(6, activation='softmax')(x) 

In [ ]:
from keras.models import Model
newModel=Model(inputs=vgg19_model.input,outputs=preds)
print("layer count: %d" %len(newModel.layers))
newModel.summary()

## data augmentation:

In [ ]:
train_datagen=ImageDataGenerator(
    height_shift_range = 0.2,
    featurewise_center=True,
    featurewise_std_normalization=True,
    zoom_range = 0.4,
    horizontal_flip = True,
    fill_mode='reflect'
)
train_datagen.fit(X_train)

train_generator=train_datagen.flow(X_train, 
                                                  Y_train_cat,
                                                  #target_size=(80,80),
                                                  #color_mode='rgb',
                                                  batch_size=16,
                                                  #class_mode='categorical',
                                                  shuffle=True
                                                 )

In [ ]:
val_datagen=ImageDataGenerator()
val_generator=val_datagen.flow(X_val, 
                                                  Y_val_cat,
                                                  #target_size=(80,80),
                                                  #color_mode='rgb',
                                                  batch_size=16,
                                                  #class_mode='categorical',
                                                  shuffle=True
                                                 )

In [ ]:
newModel.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#check n of steps per epoch:
train_generator.n//train_generator.batch_size

#### Early stopping:

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy')>0.95):
            print('\nReached 95+%, so canceling training')
            self.model.stop_training = True

In [ ]:
history = newModel.fit_generator(generator= train_generator,
                                 steps_per_epoch=100,
                                 validation_data = val_generator,
                                 epochs=5, 
                                 callbacks=myCallback())

#### evaluate model (accuracy + confusion matrices)

In [ ]:
newModel.evaluate(X_val, Y_val_cat)

In [ ]:
test_pred = newModel.predict(X_val)
test_predicted = np.argmax(test_pred, axis = 1)
test_true = np.argmax(Y_val_cat, axis = 1)
confmx = confusion_matrix(test_predicted, test_true)
f, ax = plt.subplots(figsize = (8,8))
sns.heatmap(confmx, annot=True, fmt='.1f', ax = ax)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show();